# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext version_information
%version_information numpy, unityagents, torch, matplotlib, pandas, statsmodels, version_information

Software,Version
Python,3.6.10 64bit [GCC 4.2.1 Compatible Apple LLVM 11.0.0 (clang-1100.0.33.16)]
IPython,7.15.0
OS,Darwin 19.5.0 x86_64 i386 64bit
numpy,1.19.0
unityagents,0.4.0
torch,1.5.1
matplotlib,3.2.2
pandas,1.0.5
statsmodels,The 'statsmodels' distribution was not found and is required by the application
version_information,1.0.3


In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while False:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [6]:
%load_ext autoreload
%autoreload 2
from PPO.agent import Agent, compute_advantages, compute_gae
agent = Agent(state_size, action_size, seed=0)

In [7]:
from collections import deque
from matplotlib import pyplot as plt
import pandas as pd
import torch
import time

env_info = env.reset(train_mode=True)[brain_name]

In [8]:
def plot_results(results, rolling_window=100):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(results)), results)
    rolling_mean = pd.Series(scores).rolling(rolling_window).mean()
    plt.plot(rolling_mean);
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
from collections import deque
import pickle
import torch
from PPO.utils import Trajectory

episodes = 3000
rand_seed = 42
print_every = 30

scores = []
entropies = []
scores_std = []
scores_avg = []
scores_window = deque(maxlen=100)

SOLVED = False

print('\nNN ARCHITECURES:')
print(agent.policy.actor_body)
print(agent.policy.critic_body)

print('\nTRAINING:')
time_start = time.time()

for episode in range(1, episodes+1):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    trject = Trajectory()
    entropy = 0
    while True:
        actions, log_probs, values, entropy_ = agent.act(states)
        env_info = env.step(actions.cpu().numpy())[brain_name]
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                         # see if episode finished
        trject.add(states, rewards, log_probs, actions, values, dones)
        states = next_states
        entropy += entropy_
        if np.any(dones):
            break
            
    _, _, next_value, _ = agent.act(next_states)
    returns = compute_gae(next_value, trject)
    advantages = compute_advantages(returns, trject['value'])
    agent.step(trject, returns, advantages)
    
    scores.append(trject.score)
    entropies.append(entropy)
    scores_window.append(trject.score)
    scores_avg.append(np.mean(scores_window))
    scores_std.append(np.std(scores_window))
    
    print('\rEpisode {}\tScore: {:.2f}'.format(episode, trject.score), end="")
    if episode % print_every == 0:
        torch.save(agent.policy.state_dict(), 'policy.pth')
        s = (int)(time.time() - time_start)
        print('\r*** Episode {}\tAverage Score: {:.2f}\t σ:{:2f}\t Time: {:02}:{:02}:{:02} *** '\
              .format(episode, np.mean(scores_window), np.std(scores_window), s//3600, s%3600//60, s%60)) 
    if np.mean(scores_window)>=30.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scores_window)))
        torch.save(agent.policy.state_dict(), f'final_policy.pth')
        break


NN ARCHITECURES:
Actor(
  (fc1): Linear(in_features=33, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=4, bias=True)
)
Critic(
  (fcs1): Linear(in_features=33, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)

TRAINING:
*** Episode 30	Average Score: 0.68	 σ:0.610295	 Time: 00:01:28 *** 
Episode 54	Score: 0.30

In [ ]:
plot_results(scores)